In [2]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/swift.jsonl

In [3]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
ls = []

with open('swift.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

100000it [00:04, 21194.89it/s]


99582

In [5]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [6]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [7]:
!mkdir mixtral-swift
# !rm -rf mixtral-swift/*.json

In [8]:
def answer(q, i):
    filename = f'mixtral-swift/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Swift problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Swift snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [9]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [10]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [16]:
answer(*urls[2])

In [17]:
with open('mixtral-swift/2.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

The `SCCameraViewController` class provided above is a part of a simple camera application that allows users to take photos. The application displays a camera preview using `SCPreviewView`, and users can take pictures by tapping the capture button. The `captureStillImage` method of `SCCameraController` is called when the capture button is tapped, which captures a still image and calls the `imageAvailableHandler` closure with the captured image and any potential error.

However, the current implementation does not handle any errors that might occur during image capturing. In order to improve the robustness and user experience, implement error handling by:

1. Displaying a UIAlertController with an error message if `imageAvailableHandler` receives an error.
2. Dismissing the SCCameraViewController after the user has seen the error message.

**[Solution]**

To implement error handling in the `capture` IBAction method, first, add a function for displaying alerts

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 13%|█████████████▎                                                                                     | 13357/99582 [8:47:58<48:16:49,  2.02s/it]